### Notebook to parse html files to produce cleaned text of SST decisions

Author: Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). 

Dataset & Code to be cited as: 

    Sean Rehaag, "Social Security Tribunal Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/sst>.

To load data, see [load_and_analyze_sst_cases.ipynb](https://github.com/Refugee-Law-Lab/sst_bulk_data/blob/master/load_and_analyze_sst_cases.ipynb)

### Notes:

Notes:

(1) Data Source: [Social Security Tribunal of Canada](https://www.sst-tss.gc.ca).

(2) Unofficial Data: The data are unofficial reproductions of materials on the Social Security Tribunal website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Social Security Tribunal of Canada.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Social Security Tribunal of Canada website's [Terms of Use](https://www.sst-tss.gc.ca/en/terms-and-conditions).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/SST/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/sst_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/sst_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/sst_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')
out_path_HF = pathlib.Path('d:/AI-Projects/canadian-legal-data/SST/train.parquet')

# set years sought
start_year = 2001 
end_year = 2024

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)

results = []

df


28073


100%|██████████| 28073/28073 [02:34<00:00, 182.26it/s]


,case_citation,case_citation2,case_year,case_name,case_language,case_decision_date,scraped_link,scraped_timestamp,scraped_status_code,referrer_main_source,referrer_sub_source,referrer_file,referrer_timestamp,case_html
0,2013 SSTAD 1,,2013,K.O. v. Minister of Human Resources and Skills...,en,2013-05-14,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26T07:53:08.592343,200,SR-Local,sst_batch_cpp_jul23.jsonl,C:/Scraping/QUEUES/sst_batch_cpp_jul23.jsonl,2023-07-24T17:29:23.497800,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ..."
1,2013 SSTAD 10,,2013,S. H. v. Minister of Human Resources and Skill...,en,2013-11-27,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26T07:52:25.832988,200,SR-Local,sst_batch_cpp_jul23.jsonl,C:/Scraping/QUEUES/sst_batch_cpp_jul23.jsonl,2023-07-24T17:29:23.497800,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ..."
2,2013 SSTAD 11,,2013,M. H. v. Minister of Human Resources and Skill...,en,2013-11-27,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26T07:52:20.338058,200,SR-Local,sst_batch_cpp_jul23.jsonl,C:/Scraping/QUEUES/sst_batch_cpp_jul23.jsonl,2023-07-24T17:29:23.497800,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ..."
3,2013 SSTAD 12,,2013,L. L. and Minister of Human Resources and Skil...,en,2013-12-31,https://decisions.sst-tss.gc.ca/sst-tss/cpp-rp...,2023-07-27T08:33:30.119367,200,SR-Local,sst_batch_other_cpp_jul23.jsonl,C:/Scraping/QUEUES/sst_batch_other_cpp_jul23.j...,2023-07-24T18:04:02.219239,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ..."
4,2013 SSTAD 13,,2013,R. S. v. Minister of Human Resources and Skill...,en,2013-12-17,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26T07:52:14.077767,200,SR-Local,sst_batch_cpp_jul23.jsonl,C:/Scraping/QUEUES/sst_batch_cpp_jul23.jsonl,2023-07-24T17:29:23.497800,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28068,2024 TSS 89,,2024,Commission de l’assurance-emploi du Canada c BB,fr,2024-01-29,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12T06:53:31.786288,200,SR-Local,sst_ei_update_apr24.jsonl,C:/Scraping/QUEUES/sst_ei_update_apr24.jsonl,2024-04-11T16:04:04.709065,"<!DOCTYPE html>\n<html lang=""fr"">\n\n<head>\n ..."
28069,2024 TSS 9,,2024,AP c Commission de l’assurance-emploi du Canada,fr,2024-01-04,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12T06:55:04.992982,200,SR-Local,sst_ei_update_apr24.jsonl,C:/Scraping/QUEUES/sst_ei_update_apr24.jsonl,2024-04-11T16:04:04.709065,"<!DOCTYPE html>\n<html lang=""fr"">\n\n<head>\n ..."
28070,2024 TSS 90,,2024,SB c Commission de l’assurance-emploi du Canada,fr,2024-01-30,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12T06:53:25.052075,200,SR-Local,sst_ei_update_apr24.jsonl,C:/Scraping/QUEUES/sst_ei_update_apr24.jsonl,2024-04-11T16:04:04.709065,"<!DOCTYPE html>\n<html lang=""fr"">\n\n<head>\n ..."
28071,2024 TSS 92,,2024,LB c Ministre de l’Emploi et du Développement ...,fr,2024-01-30,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2024-04-12T18:56:02.047269,200,SR-Local,sst_cpp_update_apr24.jsonl,C:/Scraping/QUEUES/sst_cpp_update_apr24.jsonl,2024-04-12T10:57:00.083539,"<!DOCTYPE html>\n<html lang=""fr"">\n\n<head>\n ..."


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = ciation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Run this in parallel to speed up

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

# Convert the pandas DataFrame to a Dask DataFrame # HAD TO INCREASE PARTITIONS DUE TO MEMORY LIMITS
ddf = dd.from_pandas(df, npartitions=50) 

# Apply the function in parallel
ddf['text'] = ddf.html.apply(get_text)

# Convert the Dask DataFrame back to a pandas DataFrame
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        df = ddf.compute()


[                                        ] | 0% Completed | 645.10 us

d:\AI-Projects\venv311\Lib\site-packages\dask\dataframe\core.py:4378: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('html', 'object'))

  warnings.warn(meta_warning(meta))


[########################################] | 100% Completed | 85.09 s


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 28072/28072 [00:20<00:00, 1348.07it/s]


In [7]:
# specific cleaning for SST:

# fix the weird footnote references
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Return to footnote \d+ referrer\n', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Return to footnote \d+ referrer', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Return to footnote \d+\n', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Return to footnote \d+', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Retour à la référence de la note de bas de page \d+ referrer\n', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Retour à la référence de la note de bas de page \d+ referrer', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Retour à la référence de la note de bas de page \d+\n', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'Retour à la référence de la note de bas de page \d+', '', x))

# remove the weird charater "\ufeff\n"
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'\ufeff\n', '', x))
df['unofficial_text'] = df['unofficial_text'].apply(lambda x: re.sub(r'\ufeff ', '', x))

# Formatting of the section headings could use some work


In [8]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# rename decision_date to document_date
df = df.rename(columns={'decision_date': 'document_date'})

# add dataset column
df['dataset'] = 'SST'

# add other column
df['other'] = ''

# reorder columns
list_cols = ['citation',
             'citation2', 
             'dataset', 
             'year', 
             'name', 
             'language', 
             'document_date', 
             'source_url', 
             'scraped_timestamp', 
             'unofficial_text',
             'other']

df = df[list_cols]

# reindex
df = df.reset_index(drop=True)

In [9]:
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2013 SSTAD 1,,SST,2013,K.O. v. Minister of Human Resources and Skills...,en,2013-05-14,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26,K.O. v. Minister of Human Resources and Skills...,
1,2013 SSTAD 10,,SST,2013,S. H. v. Minister of Human Resources and Skill...,en,2013-11-27,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26,S. H. v. Minister of Human Resources and Skill...,
2,2013 SSTAD 11,,SST,2013,M. H. v. Minister of Human Resources and Skill...,en,2013-11-27,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26,M. H. v. Minister of Human Resources and Skill...,
3,2013 SSTAD 12,,SST,2013,L. L. and Minister of Human Resources and Skil...,en,2013-12-31,https://decisions.sst-tss.gc.ca/sst-tss/cpp-rp...,2023-07-27,L. L. and Minister of Human Resources and Skil...,
4,2013 SSTAD 13,,SST,2013,R. S. v. Minister of Human Resources and Skill...,en,2013-12-17,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2023-07-26,R. S. v. Minister of Human Resources and Skill...,


In [10]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
28067,2024 TSS 89,,SST,2024,Commission de l’assurance-emploi du Canada c BB,fr,2024-01-29,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12,Commission de l’assurance-emploi du Canada c B...,
28068,2024 TSS 9,,SST,2024,AP c Commission de l’assurance-emploi du Canada,fr,2024-01-04,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12,AP c Commission de l’assurance-emploi du Canad...,
28069,2024 TSS 90,,SST,2024,SB c Commission de l’assurance-emploi du Canada,fr,2024-01-30,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12,SB c Commission de l’assurance-emploi du Canad...,
28070,2024 TSS 92,,SST,2024,LB c Ministre de l’Emploi et du Développement ...,fr,2024-01-30,https://decisions.sst-tss.gc.ca/sst-tss/cppd-r...,2024-04-12,LB c Ministre de l’Emploi et du Développement ...,
28071,2024 TSS 98,,SST,2024,JC c Commission de l’assurance-emploi du Canada,fr,2024-01-31,https://decisions.sst-tss.gc.ca/sst-tss/ei-ae/...,2024-04-12,JC c Commission de l’assurance-emploi du Canad...,


### Export data

In [11]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [12]:
# export cleaned df to parquet

df.to_parquet(out_path_parquet)

In [13]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 24/24 [00:00<00:00, 26.46it/s] 


In [14]:
# Direct to HF
df.to_parquet(out_path_HF)